# Mesclagem_Dataset.ipynb

Este notebook foi criado para mesclar múltiplos bancos de dados SQLite localizados em subpastas dentro do diretório `TABELAS` 
em um único banco consolidado. A arquitetura esperada de pastas é a seguinte:

- raiz/
  - CODE/
    - IPYNB/ -> Local onde este notebook será salvo.
  - TABELAS/
    - DIA 1 20-04-24/
      - Aparelho1.db
      - Aparelho2.db
      - Aparelho3.db
      - Aparelho4.db
    - DIA 2 21-04-24/
      - Aparelho1.db
      - Aparelho2.db
      - Aparelho3.db
      - Aparelho4.db
    - JSONS/
    - SQL'S/

Objetivo: Consolidar todos os bancos SQLite de `DIA 1` e `DIA 2` em um único banco armazenado em `SQL'S`.


In [1]:
import sqlite3
import os
from pathlib import Path

# Diretório base onde os arquivos estão localizados

In [ ]:
# Ajustar caminho para localizar TABELAS no mesmo nível de CODE
current_dir = Path().resolve()  # Diretório atual onde o notebook está
base_dir = current_dir.parent / "TABELAS"  # Sobe um nível para localizar TABELAS fora de CODE
dias = ["DIA 1 20-04-24", "DIA 2 21-04-24"]
output_dir = base_dir / "SQL'S"
output_db_path = output_dir / "Consolidado.db"

# Exibir os caminhos ajustados para validação
print(f"Diretório atual: {current_dir}")
print(f"Diretório base de TABELAS: {base_dir}")
print(f"Banco consolidado será salvo em: {output_db_path}")


Diretório atual: D:\[TCC]\CODE
Diretório base de TABELAS: D:\[TCC]\TABELAS
Banco consolidado será salvo em: D:\[TCC]\TABELAS\SQL'S\Consolidado.db


# Criação da pasta de saída, se não existir

In [5]:
output_dir.mkdir(parents=True, exist_ok=True)

# Função para copiar dados de um banco para outro

    Copia todas as tabelas de um banco SQLite de origem para um banco de destino.


In [ ]:
def merge_databases(src_db_path, dest_conn):
    src_conn = sqlite3.connect(src_db_path)
    src_cursor = src_conn.cursor()
    dest_cursor = dest_conn.cursor()
    
    # Obtendo todas as tabelas no banco de origem
    src_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = src_cursor.fetchall()
    
    for (table_name,) in tables:
        # Obtendo os dados e estrutura da tabela
        src_cursor.execute(f"SELECT * FROM {table_name};")
        rows = src_cursor.fetchall()
        
        src_cursor.execute(f"PRAGMA table_info({table_name});")
        columns = src_cursor.fetchall()
        column_names = [col[1] for col in columns]
        
        # Criando a tabela no banco de destino, se necessário
        column_defs = ", ".join([f"{col[1]} {col[2]}" for col in columns])
        dest_cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({column_defs});")
        
        # Inserindo os dados
        if rows:
            placeholders = ", ".join(["?" for _ in column_names])
            insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({placeholders});"
            dest_cursor.executemany(insert_query, rows)
    
    src_conn.close()

# Criando o banco consolidado

In [7]:
with sqlite3.connect(output_db_path) as dest_conn:
    for dia in dias:
        dia_dir = base_dir / dia
        db_files = [f for f in dia_dir.glob("*.db") if f.is_file()]
        
        for db_file in db_files:
            print(f"Processando: {db_file}")
            merge_databases(db_file, dest_conn)

print(f"Banco consolidado criado em: {output_db_path}")

Banco consolidado criado em: TABELAS\SQL'S\Consolidado.db
